# Configuring pandas

In [1]:
# import numpy and pandas
import numpy as np
import pandas as pd
import os.path
from pandas.util.testing import assert_frame_equal
import csv

# used for handling dates
import datetime
from datetime import datetime, date,timedelta

# bring in matplotlib for graphics
import matplotlib.pyplot as plt
%matplotlib inline

# Set formattign options
pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_columns', 7)
pd.set_option('display.max_rows', 10)
pd.set_option('display.width', 60)

C:\Users\marma\anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  """


# Request ticker name from user and generate list of trading days

In [2]:
userTicker = input("Enter ticker name: ")
weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
print(type(userTicker))

Enter ticker name: GOLD
<class 'str'>


In [3]:
#Request the closing day name from user
daynameTemp = str(input("Ente the day name for market close ex :"))

Ente the day name for market close ex :Friday


In [4]:
# Function to select trading day and return date in correct date format 
def get_previous_byday(dayname, start_date=None):
    if start_date is None:
        start_date = datetime.today()
        day_num = start_date.weekday()
        day_num_target = weekdays.index(dayname)
        days_ago = (7 + day_num - day_num_target) % 7
    if days_ago ==0 and start_date.hour >= 13:
        return start_date
    if days_ago == 0:
        days_ago = 7
    target_date = start_date - timedelta(days=days_ago)
    return target_date

In [5]:
#Display the target close date to user
print("This is the target date:", get_previous_byday(daynameTemp))
dateText = get_previous_byday(daynameTemp)

This is the target date: 2020-05-15 16:57:02.617963


In [6]:
#Create Year, Month, Day date variable
myYear = str(dateText.year)
myMonth = str(dateText.month)
myDay = str(dateText.day)

# Input stock ticker names from a csv file

In [7]:
#CSV file is read from current working directory
#Import os to use abs file path
#import os

#user current dir
#cwd = os.getcwd()

#new_u_file = 'textfile_pharma.csv'

#complete path file
#abs_file_path = os.path.join(cwd,new_u_file)

#if os.path.isfile(abs_file_path):
#    print("File found")
#try:
#    fhand = open(abs_file_path, 'r')
#except:
#    print('File cannot be opened:', new_u_file)
    #exit()

#tickers = list()
#for line in fhand:
#    line = line.rstrip()
#    if len(line) == 0 : continue
#    tickers.append(line)

In [8]:
#print(tickers)

# Obtaining and organizing stock data from Yahoo Finance using pandas datareader

In [9]:
# import data reader package
import pandas_datareader as pdr

#def get(tickers, startdate, enddate):
#    def data(ticker):
#        return(pdr.get_data_yahoo(ticker, start=startdate, end=enddate))
#    datas=map(data,tickers)
#    return(pd.concat(datas,keys=tickers,names=['Ticker','Date']))
#all_data=get(tickers, datetime(2020,4,7), datetime(int(myYear),int(myMonth),int(myDay)))

In [10]:
# read data from Yahoo! Finance for a specific 
# stock specified by ticker and between the start and end dates
def get_stock_data(ticker, start, end):
#    # read the data
    data = pdr.get_data_yahoo(ticker, start, end)
    # rename this column
#    data.insert(0, "Ticker", ticker)
    return data

In [11]:
# request the three years of data for SAGE
start = datetime(2020, 4, 11)
end = datetime(int(myYear),int(myMonth),int(myDay))
#raw = get_stock_data(userTicker, start, end)[:5]
raw = get_stock_data(userTicker, start, end)
print(end)

2020-05-15 00:00:00


In [12]:
raw.describe()

,High,Low,Open,Close,Volume,Adj Close
count,25.000000,25.000000,25.000000,25.000000,2.500000e+01,25.000000
mean,26.742800,25.518400,25.966400,26.244000,2.003979e+07,26.244000
std,1.239928,1.368015,1.517006,1.167197,5.931074e+06,1.167197
min,24.650000,22.250000,22.500000,24.370001,1.274080e+07,24.370001
25%,25.469999,24.360001,24.500000,25.000000,1.683540e+07,25.000000
50%,27.100000,25.920000,26.469999,26.549999,1.804800e+07,26.549999
75%,27.670000,26.570000,27.219999,27.100000,2.080960e+07,27.100000
max,28.500000,27.250000,27.930000,28.190001,3.319920e+07,28.190001


# CSV file write and read for Statistical Modeling function from Victor

In [13]:
#Hard coded file name to use for csv output file
myFile = "C:\\Users\\marma\\Documents\\investments\\csv\\"+userTicker+"_"+myMonth+"_"+myDay+".csv"

In [14]:
print(myFile)

C:\Users\marma\Documents\investments\csv\GOLD_5_15.csv


In [15]:
raw.to_csv(myFile)

In [16]:
#Read back csv file data by column index
y = pd.read_csv(myFile, index_col = 0)

In [17]:
#Extract 'Adj Close' column
d = y[['Adj Close']]

In [21]:
print(d)
print(type(d))

            Adj Close
Date                 
2020-04-13  24.370001
2020-04-14  24.430000
2020-04-15  24.469999
2020-04-16  25.000000
2020-04-17  24.540001
...               ...
2020-05-11  26.020000
2020-05-12  26.040001
2020-05-13  26.120001
2020-05-14  27.139999
2020-05-15  28.040001

[25 rows x 1 columns]
<class 'pandas.core.frame.DataFrame'>


In [19]:
# gets data for multiple stocks
# tickers: a list of stock symbols to fetch
# start and end are the start end end dates
def get_data_for_multiple_stocks(tmptickers, start, end):
    # we return a dictionary
    stocks = dict()
    # loop through all the tickers
    for ticker in tmptickers:
 #       # get the data for the specific ticker
        s = get_stock_data(ticker, start, end)
        # add it to the dictionary
        stocks[ticker] = s
    # return the dictionary
    return stocks

In [20]:
# get the data for all the stocks that we want
#print(tickers)
#raw = get_data_for_multiple_stocks(tickers, start, end)

NameError: name 'tickers' is not defined

In [ ]:
# get the data for all the stocks that we want
raw = get_data_for_multiple_stocks(['EMB','SAGE','FDVV','TLT','SAGE','IEFA','GOLD','FCNTX','JGH','AAXJ','FREL','SINA','WRB','SNP','CHL','CMA','VLO','SANM','SLVP','HDV','T'], start, end)

In [ ]:
# take a peek at the data for SAGE
raw[userTicker][:5]

# Work with a single ticker since read in list is currupting the dataframe -generate time-series chart

In [ ]:
#Open close change
raw['Change']=raw.Open - raw.Close

In [ ]:
raw['Pct_Chg']=raw.Change/raw.Open

In [ ]:
# create a time-series chart with a title and specific 
# x and y axis labels

# title is set in the .plot() method as a parameter
raw['Pct_Chg'].plot(title='Percent Change', grid=True)
#userPDTicker['Pct_Chg'].plot(grid=True)
# explicitly set the x and y axes labels after the .plot()
plt.xlabel('Time')
plt.ylabel('Percent Change');

In [ ]:
# given the dictionary of data frames,
# pivots a given column into values with column
# names being the stock symbols
def pivot_tickers_to_columns(raw, column):
    items = []
    # loop through all dictionary keys
    for key in raw:
        # get the data for the key
        data = raw[key]
        # extract just the column specified
        subset = data[["Ticker", column]]
        # add to items
        items.append(subset)
    
    # concatenate all the items
    combined = pd.concat(items)
    # reset the index
    ri = combined.reset_index()
    # return the pivot
    return ri.pivot("Date", "Ticker", column)

In [ ]:
# do the pivot
close_px = pivot_tickers_to_columns(raw, "Close")
# peek at the result
close_px[:5]

# Plotting time-series prices

In [ ]:
# plot the closing prices of SAGE
close_px[userTicker].plot();

In [ ]:
# plot the closing prices of INO
#close_px[userTicker].plot();

In [ ]:
# plot MSFT vs AAPL on the same chart
#close_px[['SAGE', userTicke]].plot();

In [ ]:
# pivot the volume data into columns
volumes = pivot_tickers_to_columns(raw, "Volume")
volumes.tail()

In [ ]:
# plot the volume for MSFT
userTicker_volume = volumes[[userTicker]]
plt.bar(INO_volume.index, userTicker_volume[userTicker])
plt.gcf().set_size_inches(15,8)

In [ ]:
# draw the price history on the top
top = plt.subplot2grid((4,4), (0, 0), rowspan=3, colspan=4)
top.plot(close_px['userTicke'].index, close_px['userTicke'], 
         label='userTicke Close')
plt.title('userTicke Close Price 2012 - 2014')
plt.legend(loc=2)

# and the volume along the bottom
bottom = plt.subplot2grid((4,4), (3,0), rowspan=1, colspan=4)
bottom.bar(userTicke_volume.index, userTicke_volume['userTicke'])
plt.title('userTicke Trading Volume 2012 - 2014')
plt.subplots_adjust(hspace=0.75)
plt.gcf().set_size_inches(15,8)

# Calculating simple daily percentage change

In [ ]:
# calculate daily percentage change
daily_pc = close_px / close_px.shift(1) - 1
daily_pc[:5]

In [ ]:
# check the percentage on 2012-01-05
close_px.loc['2017-01-05'][userTicke] / \
    close_px.loc['2017-01-04'][userTicke] -1

In [ ]:
# plot daily percentage change for INO
daily_pc[userTicke].plot();

# Calculating simple daily cumulative returns

In [ ]:
# calculate daily cumulative return
daily_cr = (1 + daily_pc).cumprod()
daily_cr[:5]

In [ ]:
# plot all the cumulative returns to get an idea 
# of the relative performance of all the stocks
daily_cr.plot(figsize=(16,6))
plt.legend(loc=2);

# Resampling data from daily to monthly returns

In [ ]:
# resample to end of month and forward fill values
monthly = close_px.asfreq('M').ffill()
monthly[:5]

In [ ]:
# calculate the monthly percentage changes
monthly_pc = monthly / monthly.shift(1) - 1
monthly_pc[:5]

In [ ]:
# calculate monthly cumulative return
monthly_cr = (1 + monthly_pc).cumprod()
monthly_cr[:5]

In [ ]:
# plot the monthly cumulative returns
monthly_cr.plot(figsize=(16,6))
plt.legend(loc=2);

# Analyzing distribution of returns

In [ ]:
# histogram of the daily percentage change for AAPL
aapl = daily_pc[userTicke]
aapl.hist(bins=50);

In [ ]:
# matrix of all stocks daily % changes histograms
daily_pc.hist(bins=50, figsize=(16,6));

# Performing moving average calculation

In [ ]:
# extract just INO close
INO_close = close_px[[userTicke]][userTicke]
# calculate the 30 and 90 day rolling means
ma_30 = INO_close.rolling(window=30).mean()
ma_90 = INO_close.rolling(window=90).mean()
# compose into a DataFrame that can be plotted
result = pd.DataFrame({'Close': INO_close, 
                       '30_MA_Close': ma_30,
                       '90_MA_Close': ma_90})
# plot all the series against each other
result.plot(title="INO Close Price")
plt.gcf().set_size_inches(15,8)
result

# Comparision of average daily returns across stocks

In [ ]:
# plot the daily percentage change of MSFT vs AAPL
plt.scatter(daily_pc[userTicke], daily_pc['SAGE'])
plt.xlabel(userTicke)
plt.ylabel('SAGE');

In [ ]:
# demonstrate perfect correlation
plt.scatter(daily_pc[userTicke], daily_pc[userTicke]);

In [ ]:
#from pandas.plotting import scatter_matrix
from pandas.plotting._misc import scatter_matrix
# plot the scatter of daily price changed for ALL stocks
scatter_matrix(daily_pc, diagonal='kde', figsize=(14,12));

# Correlation of stocks based upon daily percentage change of closing price

In [ ]:
# calculate the correlation between all the stocks relative
# to daily percentage change
corrs = daily_pc.corr()
corrs

In [ ]:
# plot a heatmap of the correlations
plt.imshow(corrs, cmap='hot', interpolation='none')
plt.colorbar()
plt.xticks(range(len(corrs)), corrs.columns)
plt.yticks(range(len(corrs)), corrs.columns)
plt.gcf().set_size_inches(12,10)

# Volatility

In [ ]:
# 75 period minimum
min_periods = 75
# calculate the volatility
vol = daily_pc.rolling(window=min_periods).std() * \
        np.sqrt(min_periods)
# plot  
vol.plot(figsize=(16, 8));

# Determining risk relative to expected returns

In [ ]:
# generate a scatter of the mean vs std of daily % change
plt.scatter(daily_pc.mean(), daily_pc.std())
plt.xlabel('Expected returns')
plt.ylabel('Risk')

# this adds fancy labels to each dot, with an arrow too
for label, x, y in zip(daily_pc.columns, 
                       daily_pc.mean(), 
                       daily_pc.std()):
    plt.annotate(
        label, 
        xy = (x, y), xytext = (30, -30),
        textcoords = 'offset points', ha = 'right', 
        va = 'bottom',
        bbox = dict(boxstyle = 'round,pad=0.5', 
                    fc = 'yellow', 
                    alpha = 0.5),
        arrowprops = dict(arrowstyle = '->', 
                          connectionstyle = 'arc3,rad=0'))

# set ranges and scales for good presentation
plt.xlim(-0.001, 0.003)
plt.ylim(0.005, 0.0600)

# set size
plt.gcf().set_size_inches(10,8)